In [11]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf

In [12]:
train = pd.read_csv('data/train_features.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test_features.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [13]:
# id랑 time label을 없앰
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600, 6])
X.shape

TensorShape([3125, 600, 6])

In [17]:
# label을 one-hot encoding (61개의 unique 한 label 존재)
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation, LSTM
from keras import optimizers

def deep_lstm():
    model = Sequential()
    model.add(LSTM(32, input_shape = (600,6), return_sequences = True))
    model.add(LSTM(64, return_sequences = True))
    model.add(LSTM(128, return_sequences = False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [19]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [20]:
model1 = deep_lstm()
model1.fit(X,y, epochs=100, batch_size=32, verbose = 1,  validation_split=0.2, callbacks = [es])

Epoch 1/100
79/79 [==============================] - 64s 742ms/step - loss: 3.0300 - accuracy: 0.4547 - val_loss: 2.6266 - val_accuracy: 0.5008
Epoch 2/100
79/79 [==============================] - 67s 854ms/step - loss: 2.6288 - accuracy: 0.4876 - val_loss: 2.5632 - val_accuracy: 0.5152
Epoch 3/100
79/79 [==============================] - 73s 924ms/step - loss: 2.4301 - accuracy: 0.4902 - val_loss: 2.7875 - val_accuracy: 0.5088
Epoch 4/100
79/79 [==============================] - 76s 962ms/step - loss: 2.3191 - accuracy: 0.4933 - val_loss: 2.3214 - val_accuracy: 0.5152
Epoch 5/100
79/79 [==============================] - 5655s 72s/step - loss: 2.1591 - accuracy: 0.5151 - val_loss: 2.3642 - val_accuracy: 0.5264
Epoch 6/100
79/79 [==============================] - 89s 1s/step - loss: 2.1157 - accuracy: 0.5051 - val_loss: 2.2621 - val_accuracy: 0.5344
Epoch 7/100
79/79 [==============================] - 85s 1s/step - loss: 1.9899 - accuracy: 0.5312 - val_loss: 2.0371 - val_accuracy: 0.536

In [21]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape
prediction=model1.predict(test_X)
submission.iloc[:,1:]=prediction
submission.to_csv('output/deepLSTM_es_submission.csv', index=False)

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras import optimizers
from keras.callbacks import EarlyStopping

def deeper_lstm():
    model = Sequential()
    model.add(LSTM(32, input_shape = (600,6), return_sequences = True))
    model.add(LSTM(64, return_sequences = True))
    model.add(LSTM(128, return_sequences = True))
    model.add(LSTM(256, return_sequences = False))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(61, activation='softmax'))
    
    #adam = optimizers.adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
    return model

In [ ]:
model1 = deeper_lstm()
hist = model1.fit(X,y, epochs=100, batch_size=32, verbose = 1,  validation_split=0.2, callbacks = [es])

Epoch 1/100
79/79 [==============================] - 345s 4s/step - loss: 2.9934 - accuracy: 0.4674 - val_loss: 2.6531 - val_accuracy: 0.5024
Epoch 2/100
79/79 [==============================] - 321s 4s/step - loss: 2.6512 - accuracy: 0.4843 - val_loss: 2.5447 - val_accuracy: 0.5104
Epoch 3/100
79/79 [==============================] - 324s 4s/step - loss: 2.5263 - accuracy: 0.4858 - val_loss: 2.4732 - val_accuracy: 0.5088
Epoch 4/100
79/79 [==============================] - 319s 4s/step - loss: 2.4655 - accuracy: 0.4875 - val_loss: 2.4136 - val_accuracy: 0.5168
Epoch 5/100
79/79 [==============================] - 305s 4s/step - loss: 2.3196 - accuracy: 0.4979 - val_loss: 2.2404 - val_accuracy: 0.5264
Epoch 6/100
79/79 [==============================] - 312s 4s/step - loss: 2.2314 - accuracy: 0.5183 - val_loss: 2.2557 - val_accuracy: 0.5168
Epoch 7/100
79/79 [==============================] - 311s 4s/step - loss: 2.0842 - accuracy: 0.5165 - val_loss: 2.2253 - val_accuracy: 0.5360
Epoch 

In [ ]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape
prediction=model1.predict(test_X)
submission.iloc[:,1:]=prediction
submission.to_csv('output/deeperLSTM_es_submission.csv', index=False)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()